# Homework 4
## Nicholas Colonna
### "I pledge my honor that I have abided by the Stevens Honor System"

In [29]:
import pandas as pd
import numpy as np
import pylab as pl
import statsmodels as sm
import matplotlib.pyplot as plt
import math
import statistics as stat

df = pd.read_csv("./Boston.csv")
print(df.head())

   Unnamed: 0     crim    zn  indus  chas    nox     rm   age     dis  rad  \
0           1  0.00632  18.0   2.31     0  0.538  6.575  65.2  4.0900    1   
1           2  0.02731   0.0   7.07     0  0.469  6.421  78.9  4.9671    2   
2           3  0.02729   0.0   7.07     0  0.469  7.185  61.1  4.9671    2   
3           4  0.03237   0.0   2.18     0  0.458  6.998  45.8  6.0622    3   
4           5  0.06905   0.0   2.18     0  0.458  7.147  54.2  6.0622    3   

   tax  ptratio   black  lstat  medv  
0  296     15.3  396.90   4.98  24.0  
1  242     17.8  396.90   9.14  21.6  
2  242     17.8  392.83   4.03  34.7  
3  222     18.7  394.63   2.94  33.4  
4  222     18.7  396.90   5.33  36.2  


(a) Based on this data set, provide an estimate for the population mean of medv. Call this estimate ˆμ.


The estimate for the population mean is calculated by simply taking the sum of all medv data points and dividing it by the number of observations. This gives us an estimate of 22.5328

In [30]:
observations = df['medv'].count()

e_mean = sum(df.medv) / observations
print(e_mean)

22.532806324110698


(b) Provide an estimate of the standard error (SE) of ˆμ. Interpret this result.
Hint: We can compute the standard error of the sample mean by dividing the sample standard deviation by the square root of the number of observations.

I used the formula for standard deviation to arrive at an estimate for SE of 0.4084

In [31]:
sum_square = 0
for val in df.medv:
    sum_square = sum_square + (val - e_mean)**2

sample_stdev = math.sqrt(sum_square/(observations))

e_stdev = sample_stdev / math.sqrt(observations)
print(e_stdev)

0.4084569346972867


(c) Now estimate the standard error of ˆμ using the bootstrap. How does this compare to your answer from (b)?
Hint: You can use the following function to generate a bootstrap sample. Your solution should generate many bootstrap samples to calculate the standard error.

The calculaed standard error using 500 bootstrap samples gives a value around 0.41, which is fairly close to our calculated SE from our data in part b.

In [38]:
 def bootstrap_sample(df, column, statistic):
        """Compute bootstrap sample of column in dataframe df applying statistic function."""
        # get a random sampling of indices
        boot_indices = np.random.choice(df.index, size=len(df), replace=True)
        # compute a sample statistic
        sample_stat = statistic(df[column].loc[boot_indices])
        return sample_stat

bootstrap_stdev = []
for i in range(0,501):
    bootstrap_stdev.append((bootstrap_sample(df, 'medv', stat.stdev))/math.sqrt(observations))

bootstrap_stdev = np.mean(bootstrap_stdev)
print(bootstrap_stdev)

0.40718471153441654


(d) Based on your bootstrap estimate from (c), provide a 95% confidence interval for the mean of medv. Compare it to the results obtained using the t test of medv. Hint: You can approximate a 95% confidence interval using the formula: ˆμ − 2SE(ˆμ), ˆμ + 2SE(ˆμ).

First, I calculated the upper and lower bound of the 95% confidence interval, then I displayed it as an inequality. I also ran a t-test on the data, and generated a confidence interval. As you can see from the results, the two methods yield very similar results.

In [39]:
upper = e_mean + 2*bootstrap_stdev
lower = e_mean - 2*bootstrap_stdev
print("Bootstrap 95% Confidence Interval: ", lower, " < μ < ", upper, sep='')

t_test = sm.stats.weightstats.DescrStatsW(df.medv)
t_test_ci = t_test.tconfint_mean()
print("   t-test 95% Confidence Interval: ", t_test_ci[0], " < μ < ", t_test_ci[1], sep='')

Bootstrap 95% Confidence Interval: 21.718436901041866 < μ < 23.34717574717953
   t-test 95% Confidence Interval: 21.729528014578587 < μ < 23.336084633642752


(e) Based on this data set, provide an estimate, ˆμmed, for the median value of medv in the population.

Since we have an even number of observations (506), the median will be the average between the two middle-most data points (253 and 254). Using the statistics median function, we obtain a median of 21.2.

In [40]:
e_median = stat.median(df.medv)
print(e_median)

21.2


(f) We now would like to estimate the standard error of ˆμmed. Unfortunately, there is no simple formula for computing the standard error of the median. Instead, estimate the standard error of the median using the bootstrap. Comment on your findings.

Using the bootstrap_sample function above, I called the function many times to generate many bootstrap samples.

In [41]:
bootstrap_median = []
for i in range(0,501):
    bootstrap_median.append(bootstrap_sample(df, 'medv', stat.median))

bootstrap__med_stdev = stat.stdev(bootstrap_median)
print(bootstrap__med_stdev)

0.36450535208225204


(g) Based on this data set, provide an estimate for the tenth percentile of medv in Boston suburbs. Call this quantity ˆμ_0.1.

Using the numpy percentile function, I was able to obtain the 10th percentile of the data, which is 12.75.

In [42]:
tenth_percentile = np.percentile(df['medv'], 10)
print(tenth_percentile)

12.75


(h) Use the bootstrap to estimate the standard error of ˆμ_0.1. Comment on your findings

I modified the bootstrap function slightly to take in one additional parameter, which indicates the percentile we want. In addition, I had to adjust the sample_stat line to include the new variable percentile. From there, it follows the same method as previous examples, creating multiple bootstrap samples and then calculating the standard deviation from the results.

In [43]:
def bootstrap_sample_percentile(df, column, statistic, percentile):
        """Compute bootstrap sample of column in dataframe df applying statistic function."""
        # get a random sampling of indices
        boot_indices = np.random.choice(df.index, size=len(df), replace=True)
        # compute a sample statistic
        sample_stat = statistic(df[column].loc[boot_indices], percentile)
        return sample_stat
    
    
bootstrap_tenth = []
for i in range(0,501):
    bootstrap_tenth.append(bootstrap_sample_percentile(df, 'medv', np.percentile, 10))

bootstrap_tenth_stdev = stat.stdev(bootstrap_tenth)
print(bootstrap_tenth_stdev)

0.5112000869934548
